In [1]:
!pip install protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-me

In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 77.0 MB/s eta 0:00:00


In [3]:
import os
import json
import torch
import faiss
import numpy as np
import time
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

# ===== CONFIG =====
INPUT_JSON = "/kaggle/input/outputic/captions_merged.json"  # Đường dẫn file JSON đầu vào
OUTPUT_FILE = "/kaggle/working/faiss_index/IC.bin"  # Đường dẫn file output
MODEL_NAME = "Qwen/Qwen3-Embedding-8B"
BATCH_SIZE = 32    
MAX_LEN = 600

# Chọn range ID cần xử lý (None = xử lý tất cả)
START_ID = 90000   # Bắt đầu từ ID nào (để None nếu muốn từ đầu)
END_ID = 180000     # Kết thúc ở ID nào (để None nếu muốn đến cuối)

os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

# ===== LOAD MODEL =====
print("📥 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

print("⚡ Loading model in FP16 (device_map=auto)...")
model = AutoModel.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=torch.float16,   
    device_map="auto"           
)
model.eval()
print("✅ Model loaded.\n")

# ===== LOAD JSON =====
print(f"📂 Loading JSON file: {INPUT_JSON}")
with open(INPUT_JSON, "r", encoding="utf-8") as f:
    data = json.load(f)

# Sắp xếp keys và filter theo range
all_keys = sorted(data.keys(), key=lambda x: int(x))
print(f"📝 Total entries in JSON: {len(all_keys)}")

# Filter by ID range
if START_ID is not None or END_ID is not None:
    filtered_keys = []
    for key in all_keys:
        key_id = int(key)
        if START_ID is not None and key_id < START_ID:
            continue
        if END_ID is not None and key_id > END_ID:
            continue
        filtered_keys.append(key)
    keys = filtered_keys
else:
    keys = all_keys

start_str = str(START_ID) if START_ID is not None else "START"
end_str = str(END_ID) if END_ID is not None else "END"
print(f"📋 Processing range: {start_str} to {end_str}")
print(f"🔢 Total IDs to process: {len(keys)}\n")

texts = [data[k] for k in keys]

# ===== EMBEDDING WITH DETAILED PROGRESS =====
print(f"{'='*70}")
print(f"🔄 Starting embedding process...")
print(f"{'='*70}\n")

emb_list = []
total_samples = len(texts)
start_time = time.time()

for i in range(0, total_samples, BATCH_SIZE):
    batch = texts[i:i+BATCH_SIZE]
    batch_keys = keys[i:i+BATCH_SIZE]
    
    inputs = tokenizer(
        batch,
        padding=True,
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt"
    ).to(model.device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        emb = outputs.last_hidden_state[:, 0]   
    
    emb = torch.nn.functional.normalize(emb, p=2, dim=1) 
    emb_list.append(emb.cpu().numpy())
    
    del inputs, outputs, emb
    torch.cuda.empty_cache()
    
    # In progress mỗi 100 IDs
    current_idx = min(i + BATCH_SIZE, total_samples)
    if (i // BATCH_SIZE) % (100 // BATCH_SIZE + 1) == 0 or current_idx == total_samples:
        elapsed_time = time.time() - start_time
        progress_pct = (current_idx / total_samples) * 100
        
        # Tính ETA
        if current_idx > 0:
            avg_time_per_sample = elapsed_time / current_idx
            remaining_samples = total_samples - current_idx
            eta_seconds = avg_time_per_sample * remaining_samples
            eta_minutes = eta_seconds / 60
            
            current_id = batch_keys[-1]
            print(f"⏳ Progress: {current_idx}/{total_samples} ({progress_pct:.1f}%) | "
                  f"Current ID: {current_id} | "
                  f"Elapsed: {elapsed_time:.1f}s | "
                  f"ETA: {eta_minutes:.1f}m")

embeddings = np.concatenate(emb_list, axis=0)
dim = embeddings.shape[1]

total_time = time.time() - start_time
print(f"\n✅ Embedding completed in {total_time:.1f}s ({total_time/60:.1f}m)")
print(f"🔢 Embedding shape: {embeddings.shape}")

# ===== BUILD FAISS INDEX =====
print(f"\n{'='*70}")
print("🔨 Building FAISS index...")
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

# ===== SAVE =====
faiss.write_index(index, OUTPUT_FILE)
print(f"✅ Index saved to: {OUTPUT_FILE}")

print(f"\n{'='*70}")
print("🏁 PROCESS COMPLETED SUCCESSFULLY!")
print(f"{'='*70}")


📥 Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Model loaded.

📂 Loading JSON file: /kaggle/input/outputic/captions_merged.json
📝 Total entries in JSON: 702648
📋 Processing range: 90000 to 180000
🔢 Total IDs to process: 90001

🔄 Starting embedding process...

⏳ Progress: 32/90001 (0.0%) | Current ID: 90031 | Elapsed: 5.5s | ETA: 257.6m
⏳ Progress: 160/90001 (0.2%) | Current ID: 90159 | Elapsed: 23.7s | ETA: 221.7m
⏳ Progress: 288/90001 (0.3%) | Current ID: 90287 | Elapsed: 43.9s | ETA: 228.1m
⏳ Progress: 416/90001 (0.5%) | Current ID: 90415 | Elapsed: 63.1s | ETA: 226.4m
⏳ Progress: 544/90001 (0.6%) | Current ID: 90543 | Elapsed: 83.5s | ETA: 229.0m
⏳ Progress: 672/90001 (0.7%) | Current ID: 90671 | Elapsed: 103.1s | ETA: 228.4m
⏳ Progress: 800/90001 (0.9%) | Current ID: 90799 | Elapsed: 140.0s | ETA: 260.2m
⏳ Progress: 928/90001 (1.0%) | Current ID: 90927 | Elapsed: 162.4s | ETA: 259.8m
⏳ Progress: 1056/90001 (1.2%) | Current ID: 91055 | Elapsed: 184.9s | ETA: 259.6m
⏳ Progress: 1184/90001 (1.3%) | Current ID: 91183 | Elapsed: 20

In [4]:
# import json
# import numpy as np
# from transformers import AutoTokenizer

# JSON_INPUT = "/kaggle/input/icasas/IC.json"
# MODEL_NAME = "Qwen/Qwen3-Embedding-8B"

# print("📥 Loading tokenizer...")
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# with open(JSON_INPUT, "r", encoding="utf-8") as f:
#     data = json.load(f)

# keys = sorted(data.keys(), key=lambda x: int(x))
# texts = [data[k] for k in keys]

# token_lengths = []

# print("🔍 Counting tokens...")
# for txt in texts:
#     tokens = tokenizer.encode(txt, add_special_tokens=True)
#     token_lengths.append(len(tokens))

# token_lengths = np.array(token_lengths)

# print("\n================== TOKEN STATS ==================")
# print(f"📊 Samples total        : {len(texts)}")
# print(f"🔢 Min tokens per sample: {token_lengths.min()}")
# print(f"📈 Max tokens per sample: {token_lengths.max()}")
# print(f"📉 Mean tokens          : {token_lengths.mean():.2f}")
# print(f"📐 Median tokens        : {np.median(token_lengths)}")
# print(f"💥 95th percentile      : {np.percentile(token_lengths, 95):.0f}")
# print("================================================\n")

# # Hiển thị top 10 mẫu dài nhất
# idx_sorted = np.argsort(token_lengths)[::-1]
# print("📌 TOP 10 LONGEST SAMPLES:\n")
# for rank in range(10):
#     idx = idx_sorted[rank]
#     print(f"#{rank+1:02d} | {keys[idx]} | {token_lengths[idx]} tokens")
